<a href="https://colab.research.google.com/github/JimKing100/Jestimate/blob/master/linear_regression/LR_Base6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [213]:
!pip install category_encoders==2.0.0
!pip install fiona
!pip install geopy

In [0]:
# Import libraries
import pandas as pd
import numpy as np
import math

import pandas_profiling

from datetime import datetime

import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.cluster import KMeans
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, classification_report

import statsmodels.api as sm

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from geopy.distance import vincenty as get_geodesic_distance

from bokeh.models import ColumnDataSource, TableColumn, DataTable
from bokeh.layouts import column
from bokeh.models.widgets import TextInput
from bokeh.plotting import curdoc

In [215]:
# Load SF real estate data - 10 years (2009-2018) of single family home sales in San Francisco downloaded from the SF MLS
# Longitude and latitude were added to the csv file prior to loading using geocoding.geo.census.gov 
df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/Jestimate/master/data/SF-SFR-Sales-Final2d.csv')

# Rename subdistr_desc to neighborhood
df = df.rename(columns={'subdist_no': 'nid', 'subdist_desc': 'neighborhood'})

# Create subset with outliers removed - 1.6%of the data
mask = (
  (df['baths'] < 6) &
  (df['beds'] < 7) &
  (df['beds'] > 0) &
  (df['lot_sf'] < 10000) &
  (df['rooms'] < 13) &
  (df['sale_price'] < 10000000) &
  (df['sf'] < 10000)
)
df = df[mask]

# Check the data
print(df.shape)
df.head(5)

(23185, 39)


,longitude,latitude,elevation,full_address,city,state,street_no,street_name,street_suffix,zip,area,district_no,district_desc,nid,neighborhood,on_market_date,cdom,orig_list_price,sale_date,sale_price,rooms,baths,beds,sf_source,sf_source_decs,sf,lot_acres,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views
0,-122.50965,37.780280,200.83,"2645 El Camino Del Mar, San Francisco, CA 94121",San Francisco,CA,2645,El Camino Del Mar,NaN,94121,1050,1,SF District 1,1050,1 - Outer Richmond,3/14/13,4,1095000,3/22/13,1260000,8,3.5,4,T,Per Tax Records,2691,NaN,0,1969,RH2,"RGLR,FNCD","PVDW,PVSW","ATCH,GARG",0,2,4BLK,1BLK,3STR,"PNRM,OCEN,PARK,GRDN"
1,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",San Francisco,CA,1278,La Playa,St,94122,2030,2,SF District 2,2030,2 - Outer Sunset,12/3/15,144,1250000,4/25/16,1075000,9,3.0,4,T,Per Tax Records,2437,0.0689,3000,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0
2,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",San Francisco,CA,1278,La Playa,St,94122,2030,2,SF District 2,2030,2 - Outer Sunset,5/18/17,36,1395000,8/17/17,1525000,9,5.0,5,D,Per Architect,2597,0.0689,3000,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0
3,-122.50924,37.777330,189.11,"590 48th Ave, San Francisco, CA 94121",San Francisco,CA,590,48th,Ave,94121,1050,1,SF District 1,1050,1 - Outer Richmond,6/27/10,42,725000,8/20/10,715000,5,1.0,2,T,Per Tax Records,1312,NaN,0,1939,RH1,RGLR,"PVDW,PVSW",GARG,0,2,3BLK,1BLK,"ATAC,2STR,FIXR","WATR,OCEN,PARK"
4,-122.50919,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121",San Francisco,CA,618,48th,Ave,94121,1050,1,SF District 1,1050,1 - Outer Richmond,6/7/12,24,1595000,7/13/12,1595000,8,4.0,4,T,Per Tax Records,3307,NaN,0,1951,NaN,RGLR,PVDW,"ATCH,GARG,ATDR,INAC",0,3,4BLK,2BLK,3STR,"PNRM,CTYL,OCEN,PARK"


In [0]:
df['sale_date'] = pd.to_datetime(df['sale_date'], infer_datetime_format=True)
df['year_sold'] = df['sale_date'].dt.year
df['month_sold'] = df['sale_date'].dt.month
df['day_sold'] = df['sale_date'].dt.day

df['on_market_date'] = pd.to_datetime(df['on_market_date'], infer_datetime_format=True)
df['year_on_market'] = df['on_market_date'].dt.year
df['month_on_market'] = df['on_market_date'].dt.month
df['day_on_market'] = df['on_market_date'].dt.day

In [217]:
hood_mult = df.groupby(['year_sold']).agg({'sale_price': ['mean']})
hood_mult = hood_mult.set_axis(hood_mult.columns.map('_'.join), axis=1, inplace=False)
hood_mult = hood_mult.reset_index()

base = hood_mult.loc[(hood_mult['year_sold'] == 2018), 'sale_price_mean'].values[0]

hood_mult['multiplier'] = base / hood_mult['sale_price_mean']
hood_mult.head(15)

,year_sold,sale_price_mean,multiplier
0,2009,9.106866e+05,2.039841
1,2010,9.690321e+05,1.917022
2,2011,9.061253e+05,2.050109
3,2012,1.016652e+06,1.827228
4,2013,1.215801e+06,1.527927
5,2014,1.373600e+06,1.352399
6,2015,1.538617e+06,1.207354
7,2016,1.614434e+06,1.150654
8,2017,1.705913e+06,1.088951
9,2018,1.857656e+06,1.000000


In [0]:
# # Create a multiplier table by year and neighborhood to adjust for annual price changes
# def mult_calc(nhood, price_mean):
#   if nhood in base_mult['nid']:
#     base = base_mult.loc[(base_mult['nid'] == nhood), 'sale_price_mean'].values[0]
#     result = base / price_mean
#   else:
#     result = base_mult['sale_price_mean'].mean() / price_mean
#   return result

# hood_mult = df.groupby(
#     ['year_sold', 'nid']
#   ).agg(
#     {
#     'sale_price': ['mean']
#     }
# )

# hood_mult = hood_mult.set_axis(hood_mult.columns.map('_'.join), axis=1, inplace=False)
# hood_mult = hood_mult.reset_index(level=[0,1])

# mask = (
#   (hood_mult['year_sold'] == 2018)
#   )

# base_mult = hood_mult[mask]

# hood_mult['multiplier'] = hood_mult.apply(lambda x: mult_calc(x['nid'], x['sale_price_mean']), axis=1)

# hood_mult.head(100)
#base_mult.loc[(base_mult['nid'] == 1010), 'sale_price_mean'].values[0]
#base_mult.loc[(base_mult['year_sold'] == 2018) & (base_mult['nid'] == 1010), 'sale_price_mean']

In [0]:
#pandas_profiling.ProfileReport(df)

In [220]:
# Train, test split on date of 01/01/2018

low_cutoff = 2008
high_cutoff = 2018
train = df[(df['year_sold'] >= low_cutoff) & (df['year_sold'] < high_cutoff)]
test  = df[df['year_sold'] >= high_cutoff]
print(train.shape)
print(test.shape)

(21024, 45)
(2161, 45)


In [221]:
# Wrangle the data for train and test
def engineer_features(X):
  
  # Fill house square foot zero values with the average house square footage by bedroom for all single family homes in SF
  averagesf_data = X.groupby('beds').sf.mean()

  # Use average sf by bedroom for each 0 value in each bedroom group 1 - 6 bedrooms
  for i in range(1, 7): 
    X.loc[(X['sf'] == 0) & (X['beds'] == i), 'sf'] = averagesf_data.loc[i]
  
  # Impute mean for null long/lat/elev based on mean of of neighborhood
  def feature_calc(feature, nid, f_dict):
    if math.isnan(feature):
      if (nid) in f_dict:
        new_feature = f_dict[nid]
        return new_feature
    else:
      return feature
    
    return feature
  
  temp = X[~X['longitude'].isna()].groupby(['nid'])['longitude'].mean()
  long_dict = dict(temp)
  X['longitude'] = X.apply(lambda x: feature_calc(x['longitude'], x['nid'], long_dict), axis=1)
  
  temp = X[~X['latitude'].isna()].groupby(['nid'])['latitude'].mean()
  lat_dict = dict(temp)
  X['latitude'] = X.apply(lambda x: feature_calc(x['latitude'], x['nid'], lat_dict), axis=1)
  
  temp = X[~X['elevation'].isna()].groupby(['nid'])['elevation'].mean()
  elev_dict = dict(temp)
  X['elevation'] = X.apply(lambda x: feature_calc(x['elevation'], x['nid'], elev_dict), axis=1)
  
  X['zip'] = X['zip'].astype(int)
  
  # Fill rooms zero values by adding beds and baths
  def room_calc(rooms_val, beds_val, baths_val):
    if rooms_val == 0:
      total = beds_val + baths_val
    else:
      total = rooms_val
      
    return total
  
  X['rooms'] = X.apply(lambda x: room_calc(x['rooms'], x['beds'], x['baths']), axis=1)
  
  # Fill baths zero values by adding beds and baths
  X.loc[(X['baths'] == 0), 'baths'] = 1
  
  # Fill lot_sf zero values by using lot_acres to calc
  def lotsf_calc(lotsf_val, lotacres_val):
    if lotsf_val == 0:
      total = lotacres_val * 43560
    else:
      total = lotsf_val
      
    return total
  
  X['lot_sf'] = X.apply(lambda x: lotsf_calc(x['lot_sf'], x['lot_acres']), axis=1)
  
  # Fill lot_acres zero values by using lot_sf to calc
  def lotacres_calc(lotacres_val, lotsf_val):
    if lotacres_val == 0:
      total = lotsf_val / 43560
    else:
      total = lotacres_val
      
    return total
  
  X['lot_acres'] = X.apply(lambda x: lotacres_calc(x['lot_acres'], x['lot_sf']), axis=1)
  
  # Engineer new feature mean_neighbor_price THIS TAKES 2+ HOURS TO RUN
  # Output was saved to nhoods files and is loaded later to save time
  nhoods = X[['sf', 'longitude', 'latitude']]
  
  def neighbor_mean(sqft, source_latitude, source_longitude):
  
    source_latlong = source_latitude, source_longitude
    source_table = X[(X['sf'] >= (sqft * .9)) & (X['sf'] <= (sqft * 1.1)) ]
    target_table = pd.DataFrame(source_table, columns = ['latitude', 'longitude', 'sale_price']) 

    def get_distance(row):
        target_latlong = row['latitude'], row['longitude']
        return get_geodesic_distance(target_latlong, source_latlong).meters

    target_table['distance'] = target_table.apply(get_distance, axis=1)

    # Get the nearest 3 locations
    nearest_target_table = target_table.sort_values(['distance'])[:3]

    return nearest_target_table['sale_price'].mean()/sqft

  #nhoods['mean_hood_ppsf'] = X.apply(lambda x: neighbor_mean(x['sf'], x['latitude'], x['longitude']), axis=1)
  #nhoods.to_csv('/content/nhoods.csv')
  
#   # Features with nan's that should be zero's
#   cols_with_nans = ['lot_sf', 'lot_acres', 'year_built', 'zoning']
#   for col in cols_with_nans:
#     X[col] = X[col].replace(np.nan, 0)
    
  # Engineer a price_sf column to use for neighborhood dataframe - drop later
  #X['price_sf'] = X['sale_price'] / X['sf']
  
  # Drop unneeded columns
  unneeded_columns = ['sale_date', 'on_market_date', 'city', 'state', 'street_no', 'street_name', 'street_suffix',
                      'day_on_market', 'month_on_market', 'year_on_market', 'day_sold', 'month_sold', 'orig_list_price', 'cdom',
                      'sf_source', 'area']
  X = X.drop(columns=unneeded_columns)
  
  return X

train = engineer_features(train)
test = engineer_features(test)

train.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-v

,longitude,latitude,elevation,full_address,zip,district_no,district_desc,nid,neighborhood,sale_price,rooms,baths,beds,sf_source_decs,sf,lot_acres,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views,year_sold
0,-122.50965,37.780280,200.83,"2645 El Camino Del Mar, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,1260000,8.0,3.5,4,Per Tax Records,2691.0,NaN,NaN,1969,RH2,"RGLR,FNCD","PVDW,PVSW","ATCH,GARG",0,2,4BLK,1BLK,3STR,"PNRM,OCEN,PARK,GRDN",2013
1,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1075000,9.0,3.0,4,Per Tax Records,2437.0,0.0689,3000.0,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0,2016
2,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1525000,9.0,5.0,5,Per Architect,2597.0,0.0689,3000.0,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0,2017
3,-122.50924,37.777330,189.11,"590 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,715000,5.0,1.0,2,Per Tax Records,1312.0,NaN,NaN,1939,RH1,RGLR,"PVDW,PVSW",GARG,0,2,3BLK,1BLK,"ATAC,2STR,FIXR","WATR,OCEN,PARK",2010
4,-122.50919,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,1595000,8.0,4.0,4,Per Tax Records,3307.0,NaN,NaN,1951,NaN,RGLR,PVDW,"ATCH,GARG,ATDR,INAC",0,3,4BLK,2BLK,3STR,"PNRM,CTYL,OCEN,PARK",2012


In [0]:
rnhoods = pd.read_csv('https://raw.githubusercontent.com/JimKing100/Jestimate/master/data/nhoods5.csv')
tnhoods = pd.read_csv('https://raw.githubusercontent.com/JimKing100/Jestimate/master/data/nhoods4.csv')
train = pd.merge(train, rnhoods[['old_index', 'mean_hood_ppsf']], left_index=True, right_on='old_index')
test = pd.merge(test, tnhoods[['old_index', 'mean_hood_ppsf']], left_index=True, right_on='old_index')
train = train.drop(columns=['old_index'])
test = test.drop(columns=['old_index'])

In [223]:
print(train.shape)
train.head()

(21024, 30)


,longitude,latitude,elevation,full_address,zip,district_no,district_desc,nid,neighborhood,sale_price,rooms,baths,beds,sf_source_decs,sf,lot_acres,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views,year_sold,mean_hood_ppsf
0,-122.50965,37.780280,200.83,"2645 El Camino Del Mar, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,1260000,8.0,3.5,4,Per Tax Records,2691.0,NaN,NaN,1969,RH2,"RGLR,FNCD","PVDW,PVSW","ATCH,GARG",0,2,4BLK,1BLK,3STR,"PNRM,OCEN,PARK,GRDN",2013,427.350427
1,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1075000,9.0,3.0,4,Per Tax Records,2437.0,0.0689,3000.0,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0,2016,444.809192
2,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1525000,9.0,5.0,5,Per Architect,2597.0,0.0689,3000.0,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0,2017,417.404698
3,-122.50924,37.777330,189.11,"590 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,715000,5.0,1.0,2,Per Tax Records,1312.0,NaN,NaN,1939,RH1,RGLR,"PVDW,PVSW",GARG,0,2,3BLK,1BLK,"ATAC,2STR,FIXR","WATR,OCEN,PARK",2010,627.540650
4,-122.50919,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,1595000,8.0,4.0,4,Per Tax Records,3307.0,NaN,NaN,1951,NaN,RGLR,PVDW,"ATCH,GARG,ATDR,INAC",0,3,4BLK,2BLK,3STR,"PNRM,CTYL,OCEN,PARK",2012,616.873299


In [224]:
print(test.shape)
test.head()

(2161, 30)


,longitude,latitude,elevation,full_address,zip,district_no,district_desc,nid,neighborhood,sale_price,rooms,baths,beds,sf_source_decs,sf,lot_acres,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views,year_sold,mean_hood_ppsf
0,-122.509190,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121-2410",94121,1,SF District 1,1050,1 - Outer Richmond,3310000,8.0,4.0,4,Per Graphic Artist,3735.000000,0.0909,3959.0,1951,NaN,0,0,"ATCH,GARG,ATDR,INAC",ONST,3,3BLK,1BLK,2STR,"PNRM,CTYL,WATR,SFRN,OCEN,PARK,GRDN,TWNP",2018,887.996430
1,-122.509090,37.759580,23.45,"1434 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1304000,5.0,2.0,3,Not Available,1515.821469,0.0606,2639.0,1900,RM1,"RGLR,LEVL",PVDW,UNCV,ONST,1,1BLK,1BLK,"SMAT,3STR","PNRM,PRTL,PARK,GRDN,HILL",2018,945.361990
2,-122.509056,37.758010,23.01,"1516 Great Highway, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1830000,7.0,3.0,4,Per Tax Records,2120.000000,0.0705,3072.0,1948,RM1,"RGLR,OCNF",0,GARG,ONST,1,2BLK,2BLK,FULL,GRDN,2018,796.383648
3,-122.509040,37.774418,47.10,"739 48th Ave, San Francisco, CA 94121-3209",94121,1,SF District 1,1050,1 - Outer Richmond,1650000,6.0,2.0,3,Not Available,1515.821469,0.0689,3000.0,1957,RH2,0,0,"ATCH,ATDR,INAC",ONST,1,0,0,0,0,2018,1078.623066
4,-122.509020,37.758636,22.70,"1492 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1300000,3.0,1.0,2,Per Appraiser,1087.000000,0.0286,1245.0,1944,RM1,0,"PVDW,PVSW",ATCH,ONST,1,1BLK,1BLK,0,0,2018,1232.750690


In [225]:
train = pd.merge(train, hood_mult[['year_sold', 'multiplier']], on='year_sold')
test = pd.merge(test, hood_mult[['year_sold', 'multiplier']], on='year_sold')
print(train.shape)
train.tail()

(21024, 31)


,longitude,latitude,elevation,full_address,zip,district_no,district_desc,nid,neighborhood,sale_price,rooms,baths,beds,sf_source_decs,sf,lot_acres,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views,year_sold,mean_hood_ppsf,multiplier
21019,-122.438538,37.710880,318.645126,"820 Rolph St, San Francisco, CA 94112",94112,10,SF District 10,10020,10 - Crocker Amazon,448000,5.0,1.0,2,Per Tax Records,1212.000000,NaN,NaN,1949,RH1,0,0,"ATCH,GARG",0,1,0,0,"ATAC,2STR",CTYL,2011,373.762376,2.050109
21020,-122.451124,37.763445,382.437614,"1560 Shrader, San Francisco, CA 94122",94122,5,SF District 5,5050,5 - Cole Valley/Parnassu,2795000,7.5,3.5,4,Not Available,1996.952897,NaN,NaN,0,NaN,0,0,0,0,2,0,0,0,0,2011,952.284188,2.050109
21021,-122.467126,37.719239,339.789592,"422 Victoria, San Francisco, CA 94122",94122,3,SF District 3,3020,3 - Merced Heights,400000,5.0,1.0,2,Not Available,1052.683672,NaN,NaN,1937,NaN,0,0,0,0,2,0,0,0,OCEN,2011,554.139243,2.050109
21022,-122.441846,37.762631,331.874286,"1-1A Vulcan, San Francisco, CA 94114",94114,5,SF District 5,5070,5 - Corona Heights,950000,11.0,3.0,4,Not Available,1996.952897,NaN,NaN,1908,NaN,0,0,0,0,0,0,0,3STR,PNRM,2011,763.049045,2.050109
21023,-122.388955,37.730513,75.325916,"52 Williams, San Francisco, CA 94127",94127,10,SF District 10,10010,10 - Bayview,210000,5.0,1.0,2,Per Tax Records,905.000000,NaN,NaN,1907,NaN,"RGLR,UPSL,FNCD","PVDW,PVSW",GARG,0,1,3BLK,1BLK,FULL,PRTL,2011,327.440147,2.050109


In [0]:
# train['adj_price_sf'] = train['mean_hood_ppsf'] * train['multiplier']
# test['adj_price_sf'] = test['mean_hood_ppsf'] * test['multiplier']
# train = train.drop(columns=['mean_hood_ppsf','multiplier'])
# test = test.drop(columns=['mean_hood_ppsf','multiplier'])
# train.head()

In [0]:
#pandas_profiling.ProfileReport(train)

In [228]:
# Train
cutoff = 2017
temp=train.copy()
train = temp[temp['year_sold'] < 2017]
val  = temp[temp['year_sold'] >= 2017]
print(train.shape, val.shape, test.shape)

(18807, 31) (2217, 31) (2161, 31)


In [229]:
# Encode and fit a linear regression model

target = 'sale_price'

features = train.columns.drop(target)
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

pipeline = make_pipeline(
  ce.OrdinalEncoder(),
  SimpleImputer(strategy='mean'), 
  LinearRegression()
)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)

# Print regression metrics for validation 
val_mse = mean_squared_error(y_val, y_pred)
val_rmse = np.sqrt(val_mse)
val_mae = mean_absolute_error(y_val, y_pred)
val_r2 = r2_score(y_val, y_pred)
print('Validation Mean Absolute Error:', val_mae)
print('Validation R^2:', val_r2)
print('\n')

ty_pred = pipeline.predict(X_test)

# Print regression metrics for test
test_mse = mean_squared_error(y_test, ty_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, ty_pred)
test_r2 = r2_score(y_test, ty_pred)
print('Test Mean Absolute Error:', test_mae)
print('Test R^2:', test_r2)
print('\n')

Validation Mean Absolute Error: 266452.3941127083
Validation R^2: 0.8116305123198359


Test Mean Absolute Error: 393551.897163717
Test R^2: 0.7404856628674243




In [230]:
X_train.head()

,longitude,latitude,elevation,full_address,zip,district_no,district_desc,nid,neighborhood,rooms,baths,beds,sf_source_decs,sf,lot_acres,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views,year_sold,mean_hood_ppsf,multiplier
0,-122.509650,37.780280,200.83,"2645 El Camino Del Mar, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,8.0,3.5,4,Per Tax Records,2691.000000,NaN,NaN,1969,RH2,"RGLR,FNCD","PVDW,PVSW","ATCH,GARG",0,2,4BLK,1BLK,3STR,"PNRM,OCEN,PARK,GRDN",2013,427.350427,1.527927
1,-122.509056,37.774548,52.78,"731 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,6.0,2.0,3,Not Available,1415.339879,NaN,NaN,1957,NaN,0,PVDW,"ATCH,ATDR,INAC",0,1,1BLK,1BLK,"JR,2STR",0,2013,601.151247,1.527927
2,-122.509056,37.774612,52.78,"727 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,6.0,2.0,3,Per Tax Records,1328.000000,NaN,NaN,1957,RH2,FNCD,PVSW,"ATCH,GARG,INAC",0,1,1BLK,0,2STR,0,2013,640.687751,1.527927
3,-122.509030,37.774420,47.10,"738 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,5.0,1.0,2,Not Available,1052.683672,NaN,NaN,1923,NaN,0,0,"GARG,ATDR,INAC",0,2,1BLK,2BLK,FULL,0,2013,760.279042,1.527927
4,-122.508606,37.778996,222.31,"8325 Geary Blvd, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,5.0,2.0,3,Per Tax Records,1702.000000,NaN,NaN,1931,NaN,0,"PVDW,PVSW","GARG,ATDR,INAC",0,3,2BLK,1BLK,SPLT,0,2013,705.052879,1.527927


In [231]:
final = test.copy()
final['prediction'] = ty_pred
final['difference'] = final['sale_price'] - final['prediction']
final['prediction'] = final['prediction'].astype(int)
final['difference'] = final['difference'].astype(int)
final.head()

,longitude,latitude,elevation,full_address,zip,district_no,district_desc,nid,neighborhood,sale_price,rooms,baths,beds,sf_source_decs,sf,lot_acres,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views,year_sold,mean_hood_ppsf,multiplier,prediction,difference
0,-122.509190,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121-2410",94121,1,SF District 1,1050,1 - Outer Richmond,3310000,8.0,4.0,4,Per Graphic Artist,3735.000000,0.0909,3959.0,1951,NaN,0,0,"ATCH,GARG,ATDR,INAC",ONST,3,3BLK,1BLK,2STR,"PNRM,CTYL,WATR,SFRN,OCEN,PARK,GRDN,TWNP",2018,887.996430,1.0,3461758,-151758
1,-122.509090,37.759580,23.45,"1434 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1304000,5.0,2.0,3,Not Available,1515.821469,0.0606,2639.0,1900,RM1,"RGLR,LEVL",PVDW,UNCV,ONST,1,1BLK,1BLK,"SMAT,3STR","PNRM,PRTL,PARK,GRDN,HILL",2018,945.361990,1.0,1773374,-469374
2,-122.509056,37.758010,23.01,"1516 Great Highway, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1830000,7.0,3.0,4,Per Tax Records,2120.000000,0.0705,3072.0,1948,RM1,"RGLR,OCNF",0,GARG,ONST,1,2BLK,2BLK,FULL,GRDN,2018,796.383648,1.0,1978800,-148800
3,-122.509040,37.774418,47.10,"739 48th Ave, San Francisco, CA 94121-3209",94121,1,SF District 1,1050,1 - Outer Richmond,1650000,6.0,2.0,3,Not Available,1515.821469,0.0689,3000.0,1957,RH2,0,0,"ATCH,ATDR,INAC",ONST,1,0,0,0,0,2018,1078.623066,1.0,2008824,-358824
4,-122.509020,37.758636,22.70,"1492 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1300000,3.0,1.0,2,Per Appraiser,1087.000000,0.0286,1245.0,1944,RM1,0,"PVDW,PVSW",ATCH,ONST,1,1BLK,1BLK,0,0,2018,1232.750690,1.0,1701688,-401688


In [232]:
final.shape

(2161, 33)

In [0]:
# Calculate the agent_pred using the mean neighborhood price/sf * square footage of the house
def agent_calc(nhood, sqft):
  a_pred = nhood_dict[nhood] * sqft
  return a_pred

# temp = final.groupby(['nid'])['price_sf'].mean()
# nhood_dict = dict(temp)
# final['agent_pred'] = final.apply(lambda x: agent_calc(x['nid'], x['sf']), axis=1)
# final['agent_diff'] = final['sale_price'] - final['agent_pred']
# final['agent_pred'] = final['agent_pred'].astype(int)
# final['agent_diff'] = final['agent_diff'].astype(int)
final['pred_percent'] = final['difference']/final['prediction']
# final['agent_percent'] = final['agent_diff']/final['agent_pred']
# final.shape

In [234]:
final.head()

,longitude,latitude,elevation,full_address,zip,district_no,district_desc,nid,neighborhood,sale_price,rooms,baths,beds,sf_source_decs,sf,lot_acres,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views,year_sold,mean_hood_ppsf,multiplier,prediction,difference,pred_percent
0,-122.509190,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121-2410",94121,1,SF District 1,1050,1 - Outer Richmond,3310000,8.0,4.0,4,Per Graphic Artist,3735.000000,0.0909,3959.0,1951,NaN,0,0,"ATCH,GARG,ATDR,INAC",ONST,3,3BLK,1BLK,2STR,"PNRM,CTYL,WATR,SFRN,OCEN,PARK,GRDN,TWNP",2018,887.996430,1.0,3461758,-151758,-0.043838
1,-122.509090,37.759580,23.45,"1434 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1304000,5.0,2.0,3,Not Available,1515.821469,0.0606,2639.0,1900,RM1,"RGLR,LEVL",PVDW,UNCV,ONST,1,1BLK,1BLK,"SMAT,3STR","PNRM,PRTL,PARK,GRDN,HILL",2018,945.361990,1.0,1773374,-469374,-0.264679
2,-122.509056,37.758010,23.01,"1516 Great Highway, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1830000,7.0,3.0,4,Per Tax Records,2120.000000,0.0705,3072.0,1948,RM1,"RGLR,OCNF",0,GARG,ONST,1,2BLK,2BLK,FULL,GRDN,2018,796.383648,1.0,1978800,-148800,-0.075197
3,-122.509040,37.774418,47.10,"739 48th Ave, San Francisco, CA 94121-3209",94121,1,SF District 1,1050,1 - Outer Richmond,1650000,6.0,2.0,3,Not Available,1515.821469,0.0689,3000.0,1957,RH2,0,0,"ATCH,ATDR,INAC",ONST,1,0,0,0,0,2018,1078.623066,1.0,2008824,-358824,-0.178624
4,-122.509020,37.758636,22.70,"1492 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1300000,3.0,1.0,2,Per Appraiser,1087.000000,0.0286,1245.0,1944,RM1,0,"PVDW,PVSW",ATCH,ONST,1,1BLK,1BLK,0,0,2018,1232.750690,1.0,1701688,-401688,-0.236053


In [235]:
pred_five_percent = (final['pred_percent'][(final['pred_percent'] >= -.05) & (final['pred_percent'] <= .05)].count())/final.shape[0]
#agent_five_percent = (final['agent_percent'][(final['agent_percent'] >= -.05) & (final['agent_percent'] <= .05)].count())/final.shape[0]
print('Prediction Error Within 5% -', pred_five_percent)
#print('Agent Error Within 5% -', agent_five_percent)

Prediction Error Within 5% - 0.12262841277186488


In [0]:
# mae = np.sum(np.absolute((final['agent_diff'])))/final['agent_diff'].count()
# print(mae)

In [0]:
# data = final[['full_address', 'neighborhood', 'nid', 'sale_price', 'prediction', 'difference']]
# data = data.drop_duplicates()

# data.head()

In [0]:
# data.shape

In [0]:
# data.to_csv('/content/display_data.csv')